# Setup
### Runs all initialization

In [ ]:
### This cell is uneditable ###
import json, random
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.display import JSON

# load review data:
reviews_file = open("cafe/reviews.json")
reviews = json.load(reviews_file)

# split into training and testing data
random.shuffle(reviews)
training_data, testing_data = train_test_split(reviews)

# split testing data into 5 simulation runs
inputs = np.array_split(testing_data, 5)

JSON([len(training_data), len(testing_data)])

## Evaluation

In [ ]:
import random

# Your training code goes here:
def train():
    print("do something")

# Your classifier code goes here:
def classify(review):
    output = {
        'review': review,
        'inputText': review['review'],
        'realLabel': review['rating'],
        'classifierLabel': random.randint(0,1),
        'confidence': random.uniform(0,1)
    }
    return output

In [ ]:
### This cell is uneditable ###
from IPython.display import JSON

outputs = []
for run in inputs:
    output = []
    for review in run:
        output.append(classify(review))
    outputs.append(output)

JSON(outputs)